In [1]:
import requests
import json
import pandas as pd
import random
import uuid
import numpy as np

In [2]:
post_data = {
    "limit": 2000
}

In [3]:
r = requests.post("http://13.126.59.19:20021/api/search/VendorProducts",data=json.dumps(post_data))

In [4]:
result = json.loads(r.text)

In [5]:
df = pd.io.json.json_normalize(result['products'])

In [6]:
product_ids = df['product.id'].values.tolist()

In [115]:
product_ids =list (set(product_ids))

In [116]:
scores = {'like':3, 'view':1, 'cart':5, 'purchase':8}

In [117]:
scores

{'like': 3, 'view': 1, 'cart': 5, 'purchase': 8}

In [128]:
def create_user(userId, items):
    nLikes = random.randint(0,30)
    nViews = random.randint(150,1000)
    nCart = random.randint(0,7)
    nPurchase = random.randint(0,3)
    cartItems = random.sample(items, nCart)
    purchaseItems = []
    try:
        purchaseItems = random.sample(cartItems, nPurchase)
    except:
        pass
    user = {
        'userId': userId,
        'like': random.sample(items, nLikes),
        'view': random.sample(items, nViews),
        'cart': cartItems,
        'purchase': purchaseItems
    }
    return user

In [129]:
def create_userdata(no_of_users, product_ids):
    users = list()
    for _ in range(no_of_users):
        users.append(create_user(str(uuid.uuid4()), product_ids))
    return users

In [130]:
users = create_userdata(100, product_ids)

In [131]:
users_df = pd.io.json.json_normalize(users)

In [132]:
users_df.head()

,cart,like,purchase,userId,view
0,"[87cfb72c-8eea-419c-a8f0-2dccc881a823, 224f3ce...","[141662da-aa37-4116-a21d-4feaccd58e20, 5d2aac8...",[],cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b,"[e145ce76-b12d-4494-98bf-b3cc92d2012d, d3a3090..."
1,"[d930df87-d9e6-463e-ba90-729d56c94235, b103829...","[847c5132-44b4-4b6b-a5ce-438825ded343, 77503a9...",[d930df87-d9e6-463e-ba90-729d56c94235],52517827-c48e-4f07-bad4-b084d8aa1897,"[b9903893-8392-4772-9e25-413cd1800633, dd6f1e1..."
2,"[0485f15b-ac43-42f6-8e02-bf54c2ec7122, cc3a694...","[c2c75b60-795a-4f85-bac4-1cadb6e9dcb2, 61da3b7...",[cc3a6946-9de7-47b4-92d2-2fe818846a5a],0af3407d-2ddc-45f8-bde0-b199b4691e5d,"[97516f17-8ef9-4449-b68f-78224901ca2b, fdffd6a..."
3,"[174efc36-adc7-4bef-86d2-df529cee9728, 4771e44...","[c335256d-a828-4486-8706-9be79b9c11c9, eef4cd0...",[174efc36-adc7-4bef-86d2-df529cee9728],d5842ca3-56d4-41ef-8a01-dd3ae6d4630c,"[e2a5756d-9399-4b72-a7aa-aa84d7c1bf2e, 83caf40..."
4,"[0c389c13-79d7-4f2c-9d85-0c2bac5a8b52, 52dfe78...","[816652d2-55c8-4066-a753-d81186e0d144, 4f2f519...",[cdd76a15-9049-4b2c-820c-53fbd65ef068],2f2ec14b-ef62-48a9-aa57-03df8e68ca60,"[3ea2fc6a-c155-4c24-87bf-7e343e80539b, 9cd48ca..."


In [133]:
ui_mat = np.zeros((len(users), len(product_ids)))
ui_df = pd.DataFrame(ui_mat, index=users_df['userId'], columns=product_ids)

In [134]:
for user in users:
    for key in scores.keys():
        for value in user[key]:
            ui_df.loc[user['userId'],value] += scores[key]

#### User-based Recommendation Systems

In [135]:
from sklearn.metrics import pairwise_distances

In [136]:
# get cosine similarities for matrix
cosine_sim = 1-pairwise_distances(ui_df, metric="cosine")

In [137]:
pd.DataFrame(cosine_sim).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.459877,0.407417,0.393527,0.409096,0.558977,0.499804,0.573767,0.311382,0.447005,...,0.356094,0.319940,0.503511,0.438843,0.404423,0.230504,0.442037,0.187475,0.445162,0.405722
1,0.459877,1.000000,0.327659,0.331401,0.366494,0.449041,0.401221,0.476627,0.294290,0.377083,...,0.269012,0.279500,0.458750,0.385298,0.312713,0.174873,0.396412,0.141945,0.350568,0.318522
2,0.407417,0.327659,1.000000,0.337509,0.317814,0.429517,0.387258,0.429598,0.261169,0.359478,...,0.254263,0.233890,0.409711,0.318610,0.263222,0.186226,0.346919,0.163688,0.332820,0.285421
3,0.393527,0.331401,0.337509,1.000000,0.311823,0.411296,0.411340,0.471953,0.232950,0.349751,...,0.214091,0.215546,0.400567,0.357818,0.263245,0.135528,0.367876,0.122522,0.316012,0.287617
4,0.409096,0.366494,0.317814,0.311823,1.000000,0.399672,0.398336,0.440879,0.206538,0.341638,...,0.232596,0.239933,0.397872,0.334461,0.309549,0.211849,0.369668,0.182061,0.340850,0.305189


In [138]:
#get pearson similarities for ratings matrix M
pearson_sim = 1-pairwise_distances(ui_df, metric="correlation")

In [139]:
pd.DataFrame(pearson_sim).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.027297,-0.007653,-0.017858,0.007614,0.044063,0.004807,-0.008425,0.004886,-0.005759,...,0.037120,0.024330,-0.024539,-0.004550,0.026375,0.036087,-0.023497,-0.013953,0.028064,0.020286
1,0.027297,1.000000,-0.030617,-0.014336,0.037836,-0.010713,-0.032420,-0.022310,0.047531,-0.004651,...,-0.017736,0.031007,0.042534,0.020315,-0.023626,0.000488,0.020496,-0.031700,-0.018978,-0.021219
2,-0.007653,-0.030617,1.000000,0.032414,0.002490,0.015968,-0.002019,-0.042561,0.027294,0.012281,...,-0.008393,-0.005519,0.012382,-0.039858,-0.058494,0.031382,-0.012696,0.012683,-0.004073,-0.032232
3,-0.017858,-0.014336,0.032414,1.000000,0.002705,-0.002170,0.050264,0.059255,-0.004220,0.007752,...,-0.055857,-0.023641,0.010041,0.030077,-0.049501,-0.028586,0.030425,-0.035145,-0.019370,-0.020213
4,0.007614,0.037836,0.002490,0.002705,1.000000,-0.024918,0.027235,-0.004596,-0.040156,-0.006025,...,-0.031701,0.007915,0.003745,-0.006608,0.015433,0.066778,0.031875,0.038841,0.016480,0.003873


In [140]:
from sklearn.neighbors import NearestNeighbors

In [141]:
# This function finds k similar users given the user_id and ratings matrix
# Note that the similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric = 'cosine', k=4):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.loc[user_id, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if ratings.index[indices.flatten()[i]] == user_id:
            continue

        else:
            print('{0}: User {1}, with similarity of {2}'.format(i, ui_df.index[indices.flatten()[i]], similarities.flatten()[i]))

    return similarities,indices

In [149]:
user_id = 'cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b'
item_id = '157083ff-74bf-4aea-ab3e-7d115f94563d'

In [145]:
similarities,indices = findksimilarusers(user_id,ui_df, metric='cosine')

4 most similar users for User cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b:

1: User 30887377-cbf1-4286-b56c-d42c4d6a12ed, with similarity of 0.5954361387198936
2: User 6fc798f1-13d9-47f1-80f2-17cda0300849, with similarity of 0.5854642920463706
3: User 8eb7257f-3b54-4c83-a319-a11a6d2e6609, with similarity of 0.5737674091335132
4: User 9a34767f-4604-4033-b1af-b8d0a5aabd73, with similarity of 0.5701188543609923


In [146]:
similarities,indices = findksimilarusers(user_id,ui_df, metric='correlation')

4 most similar users for User cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b:

1: User 91055685-3734-4a14-9f98-8ce200d04569, with similarity of 0.059365348556774666
2: User b1a9e229-f586-465b-ae00-0423e2379f01, with similarity of 0.05610771570808015
3: User 3e5d19f4-ec68-4363-bb88-b9106857b27c, with similarity of 0.055612261299321064
4: User bc4d3eec-8f08-4a39-9719-e5cf20050cf8, with similarity of 0.05319956623887878


In [150]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = 'cosine', k=4):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id,:].mean()
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0

    for i in range(0, len(indices.flatten())):
        if ratings.index[indices.flatten()[i]] == user_id:
            continue
        else: 
            ratings_diff = ratings.loc[ratings.index[indices.flatten()[i]],item_id]-np.mean(ratings.loc[ratings.index[indices.flatten()[i]],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product

    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [152]:
predict_userbased(user_id, item_id, ui_df)

4 most similar users for User cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b:

1: User 30887377-cbf1-4286-b56c-d42c4d6a12ed, with similarity of 0.5954361387198936
2: User 6fc798f1-13d9-47f1-80f2-17cda0300849, with similarity of 0.5854642920463706
3: User 8eb7257f-3b54-4c83-a319-a11a6d2e6609, with similarity of 0.5737674091335132
4: User 9a34767f-4604-4033-b1af-b8d0a5aabd73, with similarity of 0.5701188543609923

Predicted rating for user cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b -> item 157083ff-74bf-4aea-ab3e-7d115f94563d: 1


1

#### Item-based Recommendation Systems

In [154]:
#This function finds k similar items given the item_id and ratings matrix

def findksimilaritems(item_id, ratings, metric = 'cosine', k=4):
    similarities=[]
    indices=[]
    ratings=ratings.T
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.loc[item_id, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar items for item {1}:\n'.format(k,item_id))
    for i in range(0, len(indices.flatten())):
        if ratings.index[indices.flatten()[i]] == item_id:
            continue
        else:
            print('{0}: Item {1} :, with similarity of {2}'.format(i,ratings.index[indices.flatten()[i]], similarities.flatten()[i]))
    return similarities,indices

In [155]:
similarities,indices=findksimilaritems(item_id,ui_df)

4 most similar items for item 157083ff-74bf-4aea-ab3e-7d115f94563d:

1: Item 223c795b-edc8-4e05-8fa3-25b53524ffaa :, with similarity of 0.6857843623265747
2: Item 352f1f59-2f51-46a6-93a2-f19f01dded02 :, with similarity of 0.6733237089254913
3: Item 27a8613e-c644-4751-944a-c34230ec4038 :, with similarity of 0.6676381504812229
4: Item 1950c5eb-7a2b-41b2-817f-0dafaa628f00 :, with similarity of 0.6607569627877491


In [156]:
# This function predicts the rating for specified user-item combination based on item-based approach

def predict_itembased(user_id, item_id, ratings, metric = 'cosine', k=4):
    prediction= wtd_sum =0
    similarities, indices=findksimilaritems(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    for i in range(0, len(indices.flatten())):
        if ui_df.columns[indices.flatten()[i]] == item_id:
            continue;
        else:
            product = ratings.loc[user_id,ratings.columns[indices.flatten()[i]]] * (similarities[i])
            wtd_sum = wtd_sum + product

    prediction = int(round(wtd_sum/sum_wt))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))
    return prediction

In [157]:
prediction = predict_itembased(user_id,item_id,ui_df)

4 most similar items for item 157083ff-74bf-4aea-ab3e-7d115f94563d:

1: Item 223c795b-edc8-4e05-8fa3-25b53524ffaa :, with similarity of 0.6857843623265747
2: Item 352f1f59-2f51-46a6-93a2-f19f01dded02 :, with similarity of 0.6733237089254913
3: Item 27a8613e-c644-4751-944a-c34230ec4038 :, with similarity of 0.6676381504812229
4: Item 1950c5eb-7a2b-41b2-817f-0dafaa628f00 :, with similarity of 0.6607569627877491

Predicted rating for user cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b -> item 157083ff-74bf-4aea-ab3e-7d115f94563d: 0


In [83]:
M = np.asarray([[3,7,4,9,9,7], 
                [7,0,5,3,8,8],
               [7,5,5,0,8,4],
               [5,6,8,5,9,8],
               [5,8,8,8,10,9],
               [7,7,0,4,7,8]])
M=pd.DataFrame(M, index=ui_df.index[:6], columns=ui_df.columns[:6])

In [142]:
ui_df.index[:6]

Index(['cbc2ab7a-14fb-4c47-9019-c1e4da3fee6b',
       '52517827-c48e-4f07-bad4-b084d8aa1897',
       '0af3407d-2ddc-45f8-bde0-b199b4691e5d',
       'd5842ca3-56d4-41ef-8a01-dd3ae6d4630c',
       '2f2ec14b-ef62-48a9-aa57-03df8e68ca60',
       '0af849ae-4fe2-492c-8568-4a4209e7ae50'],
      dtype='object', name='userId')

In [143]:
ui_df.columns[:6]

Index(['7780831e-f479-47d9-b056-f73911c04e05',
       '9a7e73c8-df9a-4bc8-9669-871790d11ee9',
       '157083ff-74bf-4aea-ab3e-7d115f94563d',
       '7bf3ab8c-c277-415b-bb91-3bc269cf7a4a',
       'ab2838d9-5655-4c39-a9b2-9a4ca40fd2c9',
       '6b734f0b-96e9-4324-a6a1-ff3e219cbf26'],
      dtype='object')